In [1]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def parse_rule(string, antecedents):
    
    def combine_memberships(string):
        subs = string.split("/")    
        for ant in antecedents:
            if subs[0] in ant.terms:
                if len(subs) > 1:
                    ant["{}".format(string)] = reduce(lambda x, y: fuzz.fuzzy_or(ant.universe, x, ant.universe, y)[1], 
                                              [ant[sub].mf for sub in subs])

                    return ant["{}".format(string)]
                else:
                    return ant["{}".format(string)]
    
    options = string.split("_")
    rules = reduce(lambda x, y: x & y, [combine_memberships(option) for option in options])
    return rules

In [3]:
def create_rule(persona, gb, string):
    if gb == "good":
        return ctrl.Rule(antecedent=(parse_rule(string, antecedents)), 
                         consequent=persona["{}".format(gb)])
    return ctrl.Rule(antecedent=(inv_parse_rule(string, antecedents)), 
                         consequent=persona["{}".format("bad")])

# Antecedents

In [4]:
channel = ctrl.Antecedent(np.arange(0, 1.1, 0.1), "channel")
channel["cartoon"] = fuzz.trimf(channel.universe, [0.2, 0.3, 0.4])
channel["movie"] = fuzz.trimf(channel.universe, [0.7, 0.8, 1])
channel["music"] = fuzz.trimf(channel.universe, [0.15, 0.2, 0.25])

In [5]:
means = ctrl.Antecedent(np.arange(0, 300, 5), "means")
means["low"] = fuzz.trimf(means.universe, [6, 15, 25])
means["mid"] = fuzz.trimf(means.universe, [18, 35, 80])
means["high"] = fuzz.trimf(means.universe, [60, 120, 200])

In [6]:
device = ctrl.Antecedent(np.arange(0, 2.1, 0.1), "device")
device["A"] = fuzz.trimf(device.universe, [0, 0, 0.8])
device["B"] = fuzz.trimf(device.universe, [0.5, 1 , 1.5])
device["C"] = fuzz.trimf(device.universe, [1.2, 2, 2])

In [7]:
antecedents = [channel, means, device]

# Consequent

In [8]:
output = ctrl.Consequent(np.arange(0, 1, 0.01), "output")
output["good"] = fuzz.trimf(output.universe, [0.5, 1, 1])
output["bad"] = fuzz.trimf(output.universe, [0, 0, 0.5])

# Rules

In [9]:
rule1 = create_rule(output, "good", "mid_cartoon/movie_A/B")
rule2 = create_rule(output, "good", "high_cartoon/movie_C")

In [10]:
controls = [rule1, rule2]

# Control System

In [11]:
control_system = ctrl.ControlSystem(controls)

RuntimeError: Unable to resolve rule execution order. The most likely reason is two or more rules that depend on each other.
Please check the rule graph for loops.

In [ ]:
control_simulation = ctrl.ControlSystemSimulation(control_system)

In [ ]:
control_simulation.inputs({"channel":0.29, "means":77, "device":1.4})
control_simulation.compute()

print control_simulation.output

output.view(sim=control_simulation)